In [568]:
%load_ext autoreload
%autoreload 2

In [569]:
import libitg
from libitg import Symbol, Terminal, Nonterminal, Span
from libitg import Rule, CFG
from libitg import FSA
from collections import defaultdict
import numpy as np
from collections import defaultdict

In this notebook we will discuss what you need to know and do for project 2 :)


# Lexicon

I am going to use a very simple lexicon as running example, you would instead use the translation pairs we provided, but note that

* we provided you with all translation pairs under IBM1 in both directions, thus there are many pairs in there
* you should retain only the top scoring translation pairs (this will make your lexicon smaller and your forests more manageable)
* we suggest 5 translations per source word
* you need to explicitly encode insertion and deletion in your lexicon, you can bootstrap that from alignments to NULL


In [570]:
lexicon = defaultdict(set)
lexicon['le'].update(['the', '-EPS-'])  # we will assume that `le` can be deleted
lexicon['-EPS-'].update(['a', 'the'])  # we will assume that `the` and `a` can be inserted
lexicon['e'].add('and')
lexicon['chien'].add('dog')
# lexicon['chien'].add('cat')
lexicon['noir'].update(['black', 'dark'])  
lexicon['blanc'].add('white')
lexicon['petit'].update(['small', 'little'])
lexicon['petite'].update(['small', 'little'])

# Play with the parser

You should play with the basic data structures and algorithms we provided you with, namely

* Symbol: Terminal, Nonterminal, Span
* Rule
* CFG
* FSA
* ITG-related functions
* earley parser

you need to know what they do and when to use them to obtain something you need.


# ITG

We deal with ITGs by

* constructing a source-language CFG
* parsing a source string
* projecting the resulting forest onto the target-language vocabulary through the rules of an ITG and the lexicon


**Note on performance:** we could in principle instantiate a different source-side CFG for each source sentence by constraining the lexicon to source words that occur in the sentence we are translating.
That is to say that if our sentence does not contain a word (e.g. *petite*) there is no point in including that word in the CFG.

In [571]:
# this function uses the entire lexicon
src_cfg = libitg.make_source_side_itg(lexicon)

In [572]:
print(src_cfg)

[S] ||| [X]
[X] ||| [X] [X]
[X] ||| 'le'
[X] ||| '-EPS-'
[X] ||| 'e'
[X] ||| 'chien'
[X] ||| 'noir'
[X] ||| 'blanc'
[X] ||| 'petit'
[X] ||| 'petite'


# FSA

We represent a sentence as a linear-chain FSA.

In [573]:
src_fsa = libitg.make_fsa('le chien noir')
print(src_fsa)

states=4
initial=0
final=3
arcs=3
origin=0 destination=1 label=le
origin=1 destination=2 label=chien
origin=2 destination=3 label=noir


# Forests

Forests represent sets of derivations, they are represented by CFGs where symbols have been decorated with spans.
We are working with Earley parser, which takes a CFG, an FSA, the CFG's starting symbol, and the symbol that should be used as the starting symbol of the resulting CFG.
Note that Earley takes a CFG and returns another!

The most basic forest we need is one that contains all derivations of the source.

In [574]:
forest = libitg.earley(src_cfg, src_fsa, 
                       start_symbol=Nonterminal('S'), 
                       sprime_symbol=Nonterminal("D(x)"))
print(forest)

[S]:0-3 ||| [X]:0-3
[X]:0-3 ||| [X]:0-3 [X]:3-3
[X]:0-3 ||| [X]:0-2 [X]:2-3
[X]:0-3 ||| [X]:0-1 [X]:1-3
[X]:0-3 ||| [X]:0-0 [X]:0-3
[X]:0-1 ||| [X]:0-1 [X]:1-1
[X]:0-1 ||| 'le':0-1
[X]:0-1 ||| [X]:0-0 [X]:0-1
[X]:0-0 ||| '-EPS-':0-0
[X]:0-0 ||| [X]:0-0 [X]:0-0
[X]:0-2 ||| [X]:0-2 [X]:2-2
[X]:0-2 ||| [X]:0-0 [X]:0-2
[X]:0-2 ||| [X]:0-1 [X]:1-2
[X]:3-3 ||| '-EPS-':3-3
[X]:3-3 ||| [X]:3-3 [X]:3-3
[X]:1-2 ||| 'chien':1-2
[X]:1-2 ||| [X]:1-1 [X]:1-2
[X]:1-2 ||| [X]:1-2 [X]:2-2
[X]:1-1 ||| [X]:1-1 [X]:1-1
[X]:1-1 ||| '-EPS-':1-1
[X]:1-3 ||| [X]:1-2 [X]:2-3
[X]:1-3 ||| [X]:1-1 [X]:1-3
[X]:1-3 ||| [X]:1-3 [X]:3-3
[X]:2-2 ||| [X]:2-2 [X]:2-2
[X]:2-2 ||| '-EPS-':2-2
[X]:2-3 ||| [X]:2-2 [X]:2-3
[X]:2-3 ||| 'noir':2-3
[X]:2-3 ||| [X]:2-3 [X]:3-3
[D(x)] ||| [S]:0-3


Then we need to project this forest to the target vocabulary by using the lexicon and the ITG template rules.
This is precisely the set \\(\mathcal D(x)\\) of our lecture notes.

In [575]:
projected_forest = libitg.make_target_side_itg(forest, lexicon)
print(projected_forest)

[S]:0-3 ||| [X]:0-3
[X]:0-3 ||| [X]:0-3 [X]:3-3
[X]:0-3 ||| [X]:3-3 [X]:0-3
[X]:0-3 ||| [X]:0-2 [X]:2-3
[X]:0-3 ||| [X]:2-3 [X]:0-2
[X]:0-3 ||| [X]:0-1 [X]:1-3
[X]:0-3 ||| [X]:1-3 [X]:0-1
[X]:0-3 ||| [X]:0-0 [X]:0-3
[X]:0-3 ||| [X]:0-3 [X]:0-0
[X]:0-1 ||| [X]:0-1 [X]:1-1
[X]:0-1 ||| [X]:1-1 [X]:0-1
[X]:0-1 ||| 'the':0-1
[X]:0-1 ||| '-EPS-':0-1
[X]:0-1 ||| [X]:0-0 [X]:0-1
[X]:0-1 ||| [X]:0-1 [X]:0-0
[X]:0-0 ||| 'a':0-0
[X]:0-0 ||| 'the':0-0
[X]:0-0 ||| [X]:0-0 [X]:0-0
[X]:0-2 ||| [X]:0-2 [X]:2-2
[X]:0-2 ||| [X]:2-2 [X]:0-2
[X]:0-2 ||| [X]:0-0 [X]:0-2
[X]:0-2 ||| [X]:0-2 [X]:0-0
[X]:0-2 ||| [X]:0-1 [X]:1-2
[X]:0-2 ||| [X]:1-2 [X]:0-1
[X]:3-3 ||| 'a':3-3
[X]:3-3 ||| 'the':3-3
[X]:3-3 ||| [X]:3-3 [X]:3-3
[X]:1-2 ||| 'dog':1-2
[X]:1-2 ||| [X]:1-1 [X]:1-2
[X]:1-2 ||| [X]:1-2 [X]:1-1
[X]:1-2 ||| [X]:1-2 [X]:2-2
[X]:1-2 ||| [X]:2-2 [X]:1-2
[X]:1-1 ||| [X]:1-1 [X]:1-1
[X]:1-1 ||| 'a':1-1
[X]:1-1 ||| 'the':1-1
[X]:1-3 ||| [X]:1-2 [X]:2-3
[X]:1-3 ||| [X]:2-3 [X]:1-2
[X]:1-3 ||| [X]:1-1 [X]:1-3
[X

# Dealing with a translation observation

In training, we can observe translations, thus we need to constrain \\(\mathcal D(x)\\) to the set of derivations that in addition to \\(x\\) also produce our target observation \\(y\\).

We do that by intersecting the forest for \\(\mathcal D(x)\\) with an automaton that represents \\(y\\).

That is, we first make an FSA for the target sentence:

In [576]:
tgt_fsa = libitg.make_fsa('the black dog')
print(tgt_fsa)

states=4
initial=0
final=3
arcs=3
origin=0 destination=1 label=the
origin=1 destination=2 label=black
origin=2 destination=3 label=dog


and then parse this FSA with the projected forest we got earlier from Earley.
The resulting forest will represent the set \\(\mathcal D(x,y)\\).

In [577]:
ref_forest = libitg.earley(projected_forest, tgt_fsa, 
                           start_symbol=Nonterminal("D(x)"), 
                           sprime_symbol=Nonterminal('D(x,y)'))
print(ref_forest)

[D(x)]:0-3 ||| [S]:0-3:0-3
[S]:0-3:0-3 ||| [X]:0-3:0-3
[X]:0-3:0-3 ||| [X]:0-1:0-0 [X]:1-3:0-3
[X]:0-3:0-3 ||| [X]:2-3:0-2 [X]:0-2:2-3
[X]:0-3:0-3 ||| [X]:3-3:0-1 [X]:0-3:1-3
[X]:0-3:0-3 ||| [X]:0-0:0-1 [X]:0-3:1-3
[X]:0-3:0-3 ||| [X]:0-1:0-1 [X]:1-3:1-3
[X]:0-3:0-3 ||| [X]:1-3:0-3 [X]:0-1:3-3
[X]:0-1:0-0 ||| '-EPS-':0-1:0-0
[X]:0-1:0-1 ||| [X]:0-0:0-1 [X]:0-1:1-1
[X]:0-1:0-1 ||| 'the':0-1:0-1
[X]:0-1:0-1 ||| [X]:0-1:0-0 [X]:0-0:0-1
[X]:0-1:0-1 ||| [X]:0-1:0-0 [X]:1-1:0-1
[X]:0-1:0-1 ||| [X]:1-1:0-1 [X]:0-1:1-1
[X]:1-3:0-3 ||| [X]:3-3:0-1 [X]:1-3:1-3
[X]:1-3:0-3 ||| [X]:1-1:0-1 [X]:1-3:1-3
[X]:1-3:0-3 ||| [X]:2-3:0-2 [X]:1-2:2-3
[X]:2-3:0-2 ||| [X]:2-2:0-1 [X]:2-3:1-2
[X]:2-3:0-2 ||| [X]:3-3:0-1 [X]:2-3:1-2
[X]:0-2:2-3 ||| [X]:0-1:2-2 [X]:1-2:2-3
[X]:0-2:2-3 ||| [X]:1-2:2-3 [X]:0-1:3-3
[X]:3-3:0-1 ||| 'the':3-3:0-1
[X]:0-3:1-3 ||| [X]:2-3:1-2 [X]:0-2:2-3
[X]:0-3:1-3 ||| [X]:1-3:1-3 [X]:0-1:3-3
[X]:0-3:1-3 ||| [X]:0-1:1-1 [X]:1-3:1-3
[X]:0-0:0-1 ||| 'the':0-0:0-1
[X]:1-3:1-3 ||| [X]:2-3

In [578]:
# this is just to illustrate that ref_forest accepts a single target (English) string
ref_forest_as_fsa = libitg.forest_to_fsa(ref_forest, Nonterminal('D(x,y)'))
libitg.enumerate_paths_in_fsa(ref_forest_as_fsa)

{'the black dog'}

# Length constraint

In order to learn our CRF for maximum likelihood we need to compute the gradient of the log-likelihood.

The gradient invoves two expectations (see lecture notes and project description). One expectation summarises all ways in which we can derive the joint observation \\((x, y)\\), the other summarises all ways in which we can derive the incomplete observation \\(x, n\\). The former is computed out of the forest that represents \\(\mathcal D(x, y)\\) and the latter is computed out of the forest that represents \\(\mathcal D_n(x)\\).

Here we will show you how to get \\(\mathcal D_n(x)\\) by parsing a special automaton that accepts the language \\(\Delta^n\\) where \\(\Delta\\) is the vocabulary of the target language.

In [579]:
length_fsa = libitg.LengthConstraint(4, strict=False)

This special automaton accepts strings containing 1 to 4 words, no matter which words. The label -WILDCARD- is a special label such that `x == -WILDCARD-` evaluates to `True` no matter the value of `x`.

In [580]:
print(length_fsa)

states=5
initial=0
final=1 2 3 4
arcs=4
origin=0 destination=1 label=-WILDCARD-
origin=1 destination=2 label=-WILDCARD-
origin=2 destination=3 label=-WILDCARD-
origin=3 destination=4 label=-WILDCARD-


In [581]:
target_forest = libitg.earley(projected_forest, length_fsa, 
                              start_symbol=Nonterminal("D(x)"), 
                              sprime_symbol=Nonterminal("D_n(x)"))
print(target_forest)

[D(x)]:0-4 ||| [S]:0-3:0-4
[D(x)]:0-2 ||| [S]:0-3:0-2
[D(x)]:0-3 ||| [S]:0-3:0-3
[S]:0-3:0-2 ||| [X]:0-3:0-2
[S]:0-3:0-3 ||| [X]:0-3:0-3
[S]:0-3:0-4 ||| [X]:0-3:0-4
[X]:0-3:0-3 ||| [X]:1-3:0-2 [X]:0-1:2-3
[X]:0-3:0-3 ||| [X]:2-3:0-1 [X]:0-2:1-3
[X]:0-3:0-3 ||| [X]:2-3:0-2 [X]:0-2:2-3
[X]:0-3:0-3 ||| [X]:0-0:0-1 [X]:0-3:1-3
[X]:0-3:0-3 ||| [X]:0-1:0-1 [X]:1-3:1-3
[X]:0-3:0-3 ||| [X]:0-2:0-1 [X]:2-3:1-3
[X]:0-3:0-3 ||| [X]:0-2:0-2 [X]:2-3:2-3
[X]:0-3:0-3 ||| [X]:0-1:0-0 [X]:1-3:0-3
[X]:0-3:0-3 ||| [X]:0-3:0-2 [X]:0-0:2-3
[X]:0-3:0-3 ||| [X]:0-3:0-2 [X]:3-3:2-3
[X]:0-3:0-3 ||| [X]:1-3:0-3 [X]:0-1:3-3
[X]:0-3:0-3 ||| [X]:3-3:0-1 [X]:0-3:1-3
[X]:0-3:0-4 ||| [X]:0-3:0-3 [X]:3-3:3-4
[X]:0-3:0-4 ||| [X]:2-3:0-2 [X]:0-2:2-4
[X]:0-3:0-4 ||| [X]:0-3:0-2 [X]:0-0:2-4
[X]:0-3:0-4 ||| [X]:0-2:0-1 [X]:2-3:1-4
[X]:0-3:0-4 ||| [X]:3-3:0-2 [X]:0-3:2-4
[X]:0-3:0-4 ||| [X]:1-3:0-2 [X]:0-1:2-4
[X]:0-3:0-4 ||| [X]:2-3:0-3 [X]:0-2:3-4
[X]:0-3:0-4 ||| [X]:2-3:0-1 [X]:0-2:1-4
[X]:0-3:0-4 ||| [X]:0-3:0-3 [X]:0-0

In [582]:
# this produces a super large FSA which enumerates the strings in the forest
target_forest_as_fsa = libitg.forest_to_fsa(target_forest, Nonterminal('D_n(x)'))

In [583]:
# here we get the strings in a normal python set
candidates = libitg.enumerate_paths_in_fsa(target_forest_as_fsa)
for candidate in sorted(candidates):
    print(candidate)

a black
a black dog
a dark
a dark dog
a dog
a dog black
a dog dark
black a
black a dog
black dog
black dog a
black dog the
black the
black the dog
dark a
dark a dog
dark dog
dark dog a
dark dog the
dark the
dark the dog
dog a
dog a black
dog a dark
dog black
dog black a
dog black the
dog dark
dog dark a
dog dark the
dog the
dog the black
dog the dark
the black
the black dog
the dark
the dark dog
the dog
the dog black
the dog dark


In [584]:
# Let's check how many string we got
print('Strings in D_n(x): %d' % len(candidates))
# and check whether the gold-standard reference is still in the set of candidates (it should be!)
print('Does D_n(x) contain the reference? %s' % ('yes' if 'the black dog' in candidates else 'no'))

Strings in D_n(x): 40
Does D_n(x) contain the reference? yes


# Model

Earley parsing helped us instatiated the sets which will be support to our probability distribution, in particular,

* \\(\mathcal D_n(x)\\) will be support to our joint distribution \\(P(Y, D|X=x, N=n)\\)
* \\(\mathcal D(x, y)\\) will be support to our posterior distribution \\(P(D|X=x, Y=y, N=n)\\)

and our joint distribution corresponds to 

\begin{align}
P(Y=y, D=d|X=x, N=n) &= \frac{\exp(w^\top \Phi(y, d, x, n))}{\sum_{y' \in \mathcal Y_n(x)} \sum_{d' \in \mathcal D(x, y)} \exp(w^\top \Phi(y', d', x, n))} \\
 &= \frac{\exp(w^\top \Phi(y, d, x, n))}{\sum_{d' \in \mathcal D_n(x)} \exp(w^\top \Phi(y', d', x, n))}
\end{align}

where

* \\(\Phi\\) is a feature function that maps any tuple \\(y, d, x, n\\) to \\(\mathbb R^d\\)
* \\(\theta\\) is a weight vector in \\(\mathbb R^d\\)
* \\(\mathcal Y_n(x)\\) is the set of translations of \\(x\\) that are at most \\(n\\)-words long
* and the last equality is due to the fact that there is a deterministic mapping between a derivation and the string it projects on either language, that is, \\(y' = \text{yield}_\Delta(d')\\).

In order to instantiate the joint distribution for the entire space \\(\mathcal D_n(x)\\) we must use features that can be locally assigned to the steps of a derivation. This corresponds to the idea of having *local features* or *edge potentials*:

\begin{align}
P(Y=y, D=d|X=x, N=n) 
 &= \frac{\exp \left( \sum_{r_{s,t} \in d} w^\top \phi(r, s, t, x, n) \right)}{\sum_{r_{s, t} \in d' \in \mathcal D_n(x)} \exp\left( \sum_c w^\top \phi(r, s, t, x, n) \right)} \\
\end{align}

where
* a derivation is seen as a sequence of rules decorated with spans
* \\(r_{s,t}\\) is a rule decorated with a source and a target span (in the code sometimes we call this an *edge*)
* \\(\phi\\) is a (local) feature function that maps any tuple \\((r, s, t, x, n)\\) to \\(\mathbb R^d\\)
* note that \\(\Phi(y, d, x, n) = \sum_{r_{s, t} \in d} \phi(r, s, t, x, n)\\)

The formulation in terms of local potentials make it clear that in designing feature representations for derivations we have unrestricted access to \\(x\\), \\(n\\), the rule identiy \\(r\\), the source span \\(s\\) and the target span \\(t\\), but note that because we do not have unrestricted access to \\(y\\), the information that \\(t\\) provides is rather limited.

* For example, knowing the rule `X_{1-3,1-3} -> X_{2-3,1-2} X_{1-2,2-3}` tells us that the LHS spans from 1 to 3 on the source. Suppose, we are translating \\(x\\) = `le chien noir`, then we know the the LHS projects onto `chien noir`, that it is prefixed by `BOS le` and followed by `EOS` (end-of-sentence). On the other hand, if we focus on the target span, we only know that we will produce up 2 words (because `3-1=2`) but we dont't know which words those are;
* Another rule, `X_{2-3,1-2} -> noir/black` tells us a different story, it tells us that the lexical entry `noir` has been aligned to the lexical entry `black`, it also tell us about the positions where these words are, so we could for example compute a distortion feature similar to IBM2's jump value;
* A rule, `X_{0-1,0-0} -> le/-EPS-` tells us that a deletion has happened;
* Insertions can for example appear like this `X_{1-2,1-3} -> X_{1-1,1-2} X_{1-2,2-3}`, note that the first RHS symbol has an empty source span `1-1`, but the target span is longer than 0, this means an insertion has happened;
* The rule `X_{1-3,1-3} -> X_{2-3,1-2} X_{1-2,2-3}` also tells us that we are making an inversion, note that the source spans are not adjacent, but rather flipped, e.g. 2-3 for the first RHS symbol and 1-2 for the second RHS symbol, we could have a feature capture that;

After undestanding how to use the parser to get the relevant sets, you should focus on implement a feature function that can convert the information available on an edge into a sparse feature vector.

We suggest you start with a very simple feature function that is mostly *dense*. This will make development a lot easier in the beginning. Here we show a minimal version of it.



We start with some auxiliary code

In [585]:
def get_terminal_string(symbol: Symbol):
    """Returns the python string underlying a certain terminal (thus unwrapping all span annotations)"""
    if not symbol.is_terminal():
        raise ValueError('I need a terminal, got %s of type %s' % (symbol, type(symbol)))
    return symbol.root().obj()

def get_bispans(symbol: Span):
    """
    Returns the bispans associated with a symbol. 
    
    The first span returned corresponds to paths in the source FSA (typically a span in the source sentence),
     the second span returned corresponds to either
        a) paths in the target FSA (typically a span in the target sentence)
        or b) paths in the length FSA
    depending on the forest where this symbol comes from.
    """
    if not isinstance(symbol, Span):
        raise ValueError('I need a span, got %s of type %s' % (symbol, type(symbol)))
    s, start2, end2 = symbol.obj()  # this unwraps the target or length annotation
    _, start1, end1 = s.obj()  # this unwraps the source annotation
    return (start1, end1), (start2, end2)

In [586]:
span = list(target_forest.items())[8]
symbol = span[0]
rule4 = span[1][0]
print(rule4)
print(symbol)
print(type(symbol.obj()[0])==Span)
print(get_bispans(symbol))

[X]:0-3:0-2 ||| [X]:0-1:0-0 [X]:1-3:0-2
[X]:0-3:0-2
True
((0, 3), (0, 2))


and define our prototype feature function:

In [587]:
def simple_features(edge: Rule, src_fsa: FSA, eps=Terminal('-EPS-'), 
                    sparse_del=False, sparse_ins=False, sparse_trans=False,
                   ch_en=defaultdict(float), en_ch=defaultdict(float)) -> dict:
    """
    Featurises an edge given
        * rule and spans
        * src sentence as an FSA
        * TODO: target sentence length n
        * TODO: extract IBM1 dense features
    crucially, note that the target sentence y is not available!    
    """
    fmap = defaultdict(float)
    fset = set() # stores the features we've added
    if len(edge.rhs) == 2:  # binary rule
        fmap['type:binary'] += 1.0
        fset.add('type:binary')
        # here we could have sparse features of the source string as a function of spans being concatenated
        (ls1, ls2), (lt1, lt2) = get_bispans(edge.rhs[0])  # left of RHS
        (rs1, rs2), (rt1, rt2) = get_bispans(edge.rhs[1])  # right of RHS        
        
        # TODO: double check these, assign features, add some more
        if ls1 == ls2:  # deletion of source left child
            fmap['type:deletion-slc'] += 1.0
            fset.add('type:deletion-slc')
        if rs1 == rs2:  # deletion of source right child
            fmap['type:deletion-src'] += 1.0
            fset.add('type:deletion-src')
        if ls2 == rs1:  # monotone
            fmap['type:monotone'] += 1.0
            fset.add('type:monotone')
        if ls1 == rs2:  # inverted
            fmap['type:inverted'] += 1.0
            fset.add('type:inverted')
            
        
        # source span feature of rhs
        src_span_lc = ls2 - ls1
        src_span_rc = rs2 - rs1
        fmap['span:rhs:src-lc:{}'.format(src_span_lc)] += 1.0
        fmap['span:rhs:src-rc:{}'.format(src_span_rc)] += 1.0
        fset.update({'span:rhs:src-lc:{}'.format(src_span_lc),
                     'span:rhs:src-rc:{}'.format(src_span_rc)})
        # target span feature of rhs
        tgt_span_lc = lt2 - lt1
        tgt_span_rc = rt2 - rt1
        fmap['span:rhs:tgt-lc:{}'.format(tgt_span_lc)] += 1.0
        fmap['span:rhs:tgt-rc:{}'.format(tgt_span_rc)] += 1.0
        fset.update({'span:rhs:tgt-lc:{}'.format(tgt_span_lc),
                     'span:rhs:tgt-rc:{}'.format(tgt_span_rc)})
        
    else:  # unary
        symbol = edge.rhs[0]
        if symbol.is_terminal():  # terminal rule
            fmap['type:terminal'] += 1.0
            fset.add('type:terminal')
            # we could have IBM1 log probs for the traslation pair or ins/del
            (s1, s2), (t1, t2) = get_bispans(symbol)
            src_word = src_fsa.label(s1, s2)
            tgt_word = get_terminal_string(symbol)
            if symbol.root() == eps:  # symbol.root() gives us a Terminal free of annotation
                fmap['type:deletion'] += 1.0
                fset.add('type:deletion')
                # dense versions (for initial development phase)
#                 fmap['ibm1:del:logprob'] += np.log(en_ch[src_word]['<NULL>'])
                
                # sparse version
                if sparse_del:
                    fmap['del:%s' % src_word] += 1.0
                    fset.add('del:%s' % src_word)
            else:                
                if s1 == s2:  # has not consumed any source word, must be an eps rule
                    fmap['type:insertion'] += 1.0
                    fset.add('type:insertion')
                    # dense version
#                     fmap['ibm1:ins:logprob'] += np.log(ch_en['<NULL>'][tgt_word])
                    
                    # sparse version
                    if sparse_ins:
                        fmap['ins:%s' % tgt_word] += 1.0
                        fset.add('ins:%s' % tgt_word)
                else:
                    fmap['type:translation'] += 1.0
                    fset.add('type:translation')
                    # dense version
#                     fmap['ibm1:x2y:logprob'] += np.log(ch_en[src_word][tgt_word])  # y is english word 
#                     fmap['ibm1:y2x:logprob'] += np.log(en_ch[tgt_word][src_word])
                    
                    # sparse version                    
                    if sparse_trans:
                        fmap['trans:%s/%s' % (src_word, tgt_word)] += 1.0
                        fset.add('trans:%s/%s' % (src_word, tgt_word))
        
            # source span feature of rhs
            src_span = s2 - s1
            fmap['span:rhs:src:{}'.format(src_span)] += 1.0
            fset.add('span:rhs:src:{}'.format(src_span))
            # target span feature of rhs
            tgt_span = t2 - t1
            fmap['span:rhs:tgt:{}'.format(tgt_span)] += 1.0
            fset.add('span:rhs:tgt:{}'.format(tgt_span))
                
        else:  # S -> X
            fmap['top'] += 1.0
            fset.add('top')

        # bispans of lhs of edge for source and target (source and target sentence lengths)
        if isinstance(edge.lhs.obj()[0], Span): # exclude the (Nonterminal('D(x)'), 0, 2) rules
            (s1, s2), (t1, t2) = get_bispans(edge.lhs)
            # source span feature of lhs
            src_span = s2 - s1
            fmap['span:lhs:src:{}'.format(src_span)] += 1.0
            fset.add('span:lhs:src:{}'.format(src_span))
            # target span feature of lhs
            tgt_span = t2 - t1
            fmap['span:lhs:tgt:{}'.format(tgt_span)] += 1.0
            fset.add('span:lhs:tgt:{}'.format(tgt_span))

    return fmap, fset

Now we can simply featurize edges, one at a time.

In [588]:
def featurize_edges(forest, src_fsa, 
                    sparse_del=False, sparse_ins=False, sparse_trans=False,
                    eps=Terminal('-EPS-')) -> dict:
    edge2fmap = defaultdict(float)
    fset_accum = set()
    for edge in forest:
        edge2fmap[edge], fset = simple_features(edge, src_fsa, eps, sparse_del, sparse_ins, sparse_trans)
        fset_accum.update(fset)
    return edge2fmap, fset_accum

In [678]:
edge2fmap, fset = featurize_edges(target_forest, src_fsa,
                                        sparse_del=True, sparse_ins=True, sparse_trans=True)

# print('\n'.join(sorted(list(fset))))
# # seeing features at work:
# for edge in target_forest:
#     print(edge)
#     for fmap in edge2fmap[edge]:
#         print('\t{}'.format(fmap))

Now, recall the definition of the joint distribution
\begin{align}
P(y,d|x, n) &= \frac{\exp \left( \sum_{r_{s,t} \in d} w^\top \phi(r, s, t, x, n) \right)}{Z_n(x)}
\end{align}
for convenience, let's work with log probabilities, 
\begin{align}
\log P(y,d|x,n) &= \log \exp \left( \sum_{r_{s,t} \in d} w^\top \phi(r, s, t, x, n) \right) - \log Z_n(x) \\
  &= \underbrace{\sum_{r_{s,t} \in d} \underbrace{w^\top \phi(r, s, t, x, n)}_{\text{local log-potential}}}_{\text{log of unnormalised distribution}} - \log Z_n(x)
\end{align}
Now, in the log-domain, we can compute the logarithm of unnormalised probabilities by summing along the edges in a derivation.

This is great news! We can express the log of the unnormalised distribution by decorating edges with local log-potentials, that is, 
\begin{align}
\omega(r_{s,t}) = w^\top \phi(r, s, t, x, n)
\end{align}
where \\(\omega\\) takes an edge in the hypergraph and returns its local log-potential (dot product of feature weights and feature vector).


You are ready to define a weight function now:

In [590]:
def weight_function(edge, fmap, wmap) -> float:
    # dot product of fmap and wmap  (working in log-domain)
    dot = 0.0
    for feature, value in fmap.items():
        dot += value * wmap[feature]
    return dot

In [679]:
# initial feature map has value 1.0 for all features
# wmap = {feature: 0.01 for feature in fset}
wmap = defaultdict(float)
for feature in fset:
    wmap[feature] = 0.01

edge_weights = defaultdict(float)

# seeing the dot prodcut at work
# for edge in target_forest:
#     print(edge)
#     fmap = edge2fmap[edge]
#     dot = weight_function(edge, fmap, wmap)
#     edge_weights[edge] = dot
#     print('\t{}'.format(dot))

# Tips

Working with hypergraphs can be tedious, they can be quite large, a few tips:

* do not parse very long sentences: discard training instances where either string is longer than say 10 words in development phase, for the final report you can try to stretch all the way to 30 or so;
* parse once: with a fixed lexicon and grammar, the parse forests will never change, thus produce them one at a time and pickle them to disk, so that whenever you need the forest during SGD it will be pre-computed
* featurise once: same applies to feature vectors, if you do not change your feature function, feature vectors will remain the same, thus, extract them once and pickle them to disk



# MLE

Now it's time you optimise your likelihood
\begin{align}
\mathcal L(w|x, y) &= \log P(y|x, n) \\
 &= \log \sum_{d \in \mathcal D(x, y)} P(y, d|x, n) \\
 &= \log \sum_{d \in \mathcal D(x, y)} \frac{\exp(w^\top \Phi(y, d|x, n))}{Z_n(x)} \\
 &= \log \frac{\sum_{d \in \mathcal D(x, y)} \exp(w^\top \Phi(y, d|x, n))}{Z_n(x)} \\
 &= \log \frac{Z(x, y)}{Z_n(x)} \\
 &= \log Z(x, y) - \log Z_n(x)
\end{align}

First of all, we need to efficiently compute
* \\(\log Z(x,y)\\) the sum of all unnormalised probabilities for derivations that explain the translation pair \\(x, y\\)
* and \\(\log Z_n(x)\\) the sum of all unnormalised probabilities for derivations that explain the incomplete observation \\(x, n\\)
now note that we have acyclic hypergraphs whose edges represent log-potentials for a model which is log-linear. This means that we can efficiently compute the global log-normalisers with a recursive formula that visits each node in the forest once. This recursion is called the `Inside algorithm`.

The *inside weight* of a node corresponds to the sum of the weights of all paths under this node in the forest.

\begin{equation}
    I(v) = 
    \begin{cases}
        1 \quad \text{if } v \text{ is terminal }\\
        \bigoplus_{e \in BS(v)} \omega(e) \bigotimes_{u \in tail(e)} I(u) \quad \text{ otherwise}
    \end{cases}
\end{equation}

where
* \\(u\\) and \\(v\\) are nodes (these are annotated symbols in the forest)
* \\(e\\) is an edge (this is an annotated rule in the forest)
* \\(\text{tail}(e)\\) is a sequence of children nodes (this is the RHS of an annotated rule in the forest)
* \\(BS(v)\\) corresponds to the set of edges that are incoming to v (this is the set of annotated rules for which v is LHS in the forest)
* \\(\otimes\\) corresponds to product of probabilities or equivalently sum of log-probabilities
* \\(\otimes\\) corresponds to sum of probabilities or equivalently log-of-sum-of-exponentiated-log-probabilities, i.e. \\(\log (\exp(a) + \exp(b))\\) where \\(a\\) and \\(b\\) are log-probabilities
* if \\(\otimes\\) and \\(\oplus\\) confused you, then you need to learn a little more about [semirings](http://www.aclweb.org/anthology/J/J99/J99-4004.pdf)

The inside recursion can be implemented iteratively by visiting the nodes in [top-sorted order](https://en.wikipedia.org/wiki/Topological_sorting).
              
Check the lecture notes and [slides](https://uva-slpl.github.io/nlp2/resources/slides/crf.pdf) for a pseudo-code of the Inside algorithm.

The inside at the root of the forest represents the log-normaliser of the forest.

Now you should be ready to implement topsort and the inside algorithm:

In [592]:
def top_sort(forest: CFG) -> list:
    """Returns ordered list of nodes according to topsort order in an acyclic forest"""
    S = {symbol for symbol in forest.terminals} # (Copy!) only terminals have no dependecies
    D = {symbol: {child for rule in forest.get(symbol) for child in rule.rhs}\
                 for symbol in forest.nonterminals|forest.terminals} # forest.nonterminals|forest.terminals = V
    L = list()
    while S: # while S nonempty
        u = S.pop()
        L.append(u)
        outgoing = [e for e in forest if u in e.rhs] # outgoing = FS(u)
        for rule in outgoing:
            v = rule.lhs
            D[v] = D[v] - {u}
            if len(D[v]) == 0:
                S = S | {v}
    return L

def inside_algorithm(forest: CFG, tsort: list, edge_weights: dict) -> dict:
    """Returns the inside weight of each node"""
    I = dict()
    for symbol in tsort: # symbol is v
        incoming = forest.get(symbol) # BS(v) - gets all the incoming nodes, i.e. all rules where symbol is lhs
        if len(incoming) == 0: 
            I[symbol] = 1.0 # leaves
        else:
            w = 0.0
            for edge in incoming: # edge is of type Rule
                k = edge_weights[edge]
                for child in edge.rhs: # chid in tail(e)
                    k *= I[child] # TODO: change to log-sum-exp
                w += k
            I[symbol] = w
    return I

The inside at the root of the forest represents the log-normaliser of the forest: `I[root]= `\\(\log Z_n(x)\\)

In [593]:
tsort = top_sort(target_forest)
edge_weights = {edge: weight_function(edge, edge2fmap[edge], wmap) for edge in target_forest}

I = inside_algorithm(target_forest, tsort, edge_weights)

root = Nonterminal("D_n(x)")
print(I[root])

1.4328732999168006e-10


Now, we will approach the optimisation of the log-likelihood via SGD, that is, 
we will do so by taking steps towards the steepest ascent at each training instance (or mini-batch).

Taking derivatives with respect to \\(w\\) we get

\begin{align}
\nabla_w \mathcal L(w|x, y, n) &= \mathbb E_{P(D|Y=y, X=x, N=n)}[\Phi(Y, D, X, N)] - \mathbb E_{P(Y, D|X=x, N=n)}[\Phi(Y, D, X, N)]
\end{align}

where
* the first expectation uses the forest \\(\mathcal D(x, y)\\)
* the second expectation uses the forest \\(\mathcal D_n(x)\\)

again because our CRF is edge-factored, we can compute this rather efficiently combining inside weights with *outside* weights, for which we need the *outside algorithm*. If you heard of forward-backward, then inside-outside is just a generalisation for hypergraphs (instead of graphs). You can learn more about expectations from forest from [Li and Eisner](http://www.aclweb.org/anthology/D09-1005).

Whereas the inside was defined easily on its own, the outside will be defined differently so that
\begin{align}
    I(u) \otimes O(u)
\end{align}
is proportional to the marginal probability of node \\(u\\), where the missing proportionality constant corresponds to the log-normaliser of the distribution (which is precisely the Inside weight of the root of the forest).

\begin{equation}
    O(v) = 
    \begin{cases}
        1 \quad \text{if } FS(v) =\emptyset \\
        \bigoplus_{e \in FS(v)} O(\text{head}(e)) \bigotimes_{s \in tail(e)\setminus\{v\}} I(s) \quad \text{ otherwise}
    \end{cases}
\end{equation}

where
* \\(s\\) and \\(v\\) are nodes (these are annotated symbols in the forest)
* \\(e\\) is an edge (this is an annotated rule in the forest)
* \\(\text{head}(e)\\) is a the edge's head node (this is the annotated LHS of an annotated rule in the forest)
* \\(FS(v)\\) corresponds to the set of edges that are outgoing from v (this is the set of annotated rules for which v appears in the RHS)
* \\(I(s)\\) is the inside weight of node \\(s\\)
* \\(\otimes\\) corresponds to product of probabilities or equivalently sum of log-probabilities
* \\(\otimes\\) corresponds to sum of probabilities or equivalently log-of-sum-of-exponentiated-log-probabilities, i.e. \\(\log (\exp(a) + \exp(b))\\) where \\(a\\) and \\(b\\) are log-probabilities
* if \\(\otimes\\) and \\(\oplus\\) confused you, then you need to learn a little more about [semirings](http://www.aclweb.org/anthology/J/J99/J99-4004.pdf)


Again, check the [slides](https://uva-slpl.github.io/nlp2/resources/slides/crf.pdf) for a pseu-code for the outside algorithm.

Now you should be ready to implement the *outside algorithm*.

In [594]:
def outside_algorithm(forest: CFG, tsort: list, edge_weights: dict, inside: dict, root: Symbol) -> dict:
    """Returns the outside weight of each node"""
    O = dict()
    for symbol in tsort:
        O[symbol] = 0.0
    O[root] = 1.0
    for symbol in reversed(tsort):
        incoming = forest.get(symbol)
        for edge in incoming:
            for u in edge.rhs: # u in tail(e)
                k = edge_weights[edge] * O[symbol]
                for s in edge.rhs:
                    if not u == s:
                        k *= inside[s] # TODO: change to log-sum-exp
                O[u] += k
    return O

In [595]:
O = outside_algorithm(target_forest, tsort, edge_weights, I, root=Nonterminal("D_n(x)"))

It's now time to combine inside and outside and efficiently compute the expected feature vector under each distribution.

This takes a linear pass over the edges of the forest, and [Li and Eisner](http://www.aclweb.org/anthology/D09-1005) taught us how to do that very efficiently (see figure 4, or our [slides](https://uva-slpl.github.io/nlp2/resources/slides/crf.pdf) for a clean pseudo-code).

You should be ready to implement it.

In [596]:
def expected_feature_vector(forest: CFG, inside: dict, outside: dict, edge_features: dict) -> dict:
    """Returns an expected feature vector (here a sparse python dictionary)"""
    expected_features = defaultdict(lambda:defaultdict(float))
    for rule in forest:
        k = outside[rule.lhs]
        for symbol in rule.rhs:
            k *= inside[symbol]
        for feature in edge_features[rule]:
            expected_features[rule][feature] = k * edge_features[rule][feature]
    return expected_features


# SGD

Now it's time to take gradient steps,

\begin{align}
    w = w + \delta \nabla_w \mathcal L(w|x, y, n)
\end{align}

where for a certain observation \\(x, n, y\\) we compute the expected feature vectors, subtract them, scale by a learning rate and get a parameter update.
If we have a mini-batch, we accumulate the gradients.

In the beginning of the project you will be working with the simple (mostly dense) feature function, thus this should work reasonably well. At some point you will model sparse features and then you will most likely need a [regulariser](http://www.aclweb.org/anthology/P08-1024).

To diagonose convergence we typically track log-likelihood of training data, with online learning or mini-batching is typically too expensive to go over the entire trainin set computing log-likelihood, thus you can do that on each batch individually and track both the batch log-likelihood and a running average.

Lastly, you might want to experiment with averaged parameters where in parallel to the parameters \\(w\\) that you actually use, you can keep a running average of parameters \\(w_{\text{avg}}\\), at then end of learning, this running average typically shows better performance at prediction time.

In [751]:
import progressbar
from itertools import chain
from math import nan

def update_w(wmap, expected_features_D_xy, expected_features_Dn_x, delta=0.1):
    w_new = defaultdict(float)
    nabla_total = 0.0
    for rule in chain(expected_features_D_xy, expected_features_Dn_x):
        for feature in chain(expected_features_D_xy[rule], expected_features_Dn_x[rule]):
            nabla_w = delta * (expected_features_D_xy[rule][feature] - expected_features_Dn_x[rule][feature])
            w_new[feature] = wmap[feature] + nabla_w
            nabla_total += nabla_w

    return w_new, nabla_total

def sgd_func(iters, delta, w, target_forest, ref_forest, src_fsa, sparse=False):
    """
    Performs stochastic gradient descent on the weights vector w.
    """    
    for i in range(iters):
        
        bar = progressbar.ProgressBar(max_value=13)
        bar.update(0)
        
        ### D_n(x) ###
        
        tgt_edge2fmap, _ = featurize_edges(target_forest, src_fsa,
                                           sparse_del=sparse, sparse_ins=sparse, sparse_trans=sparse)
        
        bar.update(1)

        # recompute edge weights
        tgt_edge_weights = {edge: np.exp(weight_function(edge, tgt_edge2fmap[edge], w)) 
                                for edge in target_forest}

        bar.update(2)
        
        # compute inside and outside
        tgt_tsort = top_sort(target_forest)
        root_tgt = Nonterminal("D_n(x)")
        bar.update(3)
        I_tgt = inside_algorithm(target_forest, tgt_tsort, tgt_edge_weights)
        bar.update(4)
        O_tgt = outside_algorithm(target_forest, tgt_tsort, tgt_edge_weights, I_tgt, root_tgt)
        bar.update(5)

        # compute expected features
        expected_features_Dn_x = expected_feature_vector(target_forest, I_tgt, O_tgt, tgt_edge2fmap)
        bar.update(6)

        
        ### D(x,y) ###
        
        ref_edge2fmap, _ = featurize_edges(ref_forest, src_fsa,
                                           sparse_del=sparse, sparse_ins=sparse, sparse_trans=sparse)
        bar.update(7)

        # recompute edge weights
        ref_edge_weights = {edge: np.exp(weight_function(edge, ref_edge2fmap[edge], w))
                            for edge in ref_forest}

        bar.update(8)
        
        # compute inside and outside
        tsort = top_sort(ref_forest)
        root_ref = Nonterminal("D(x,y)")
        bar.update(9)
        I_ref = inside_algorithm(ref_forest, tsort, ref_edge_weights)
        bar.update(10)
        O_ref = outside_algorithm(ref_forest, tsort, ref_edge_weights, I_ref, root_ref)
        bar.update(11)

        # compute expected features
        expected_features_D_xy = expected_feature_vector(ref_forest, I_ref, O_ref, ref_edge2fmap)
        bar.update(12)
        
        # update w
        w_new, nabla_total = update_w(w, expected_features_D_xy, expected_features_Dn_x, delta=delta)
        bar.update(13)

        w = w_new
        bar.finish()

        d = viterbi(target_forest, tgt_tsort, tgt_edge_weights, I_tgt, root_tgt) # use exp!
        candidates = write_derrivation(d)
        print("Best y = '{}'".format(candidates.pop()))
        print('P(y,d|x) = {}'.format(joint_prob(d, tgt_edge_weights, I_tgt, root_tgt))) # use

    return w

In [752]:
def joint_prob(derrivation, estimated_weights, inside, root):
    """
    Computes the joint probability of a a sentence and its derrivation.
    """
    numerator = np.exp(sum([np.log(estimated_weights[edge]) for edge in derrivation]))
    Z = inside[root]
    return numerator / Z

In [755]:
w_init = defaultdict(float)
for feature in fset:
    w_init[feature] = 0.01*np.random.uniform()
w_test = sgd_func(5, 0.01, w_init, target_forest, ref_forest, src_fsa, sparse=True)

100% (13 of 13) |#########################| Elapsed Time: 0:00:00 Time: 0:00:00
N/A% (0 of 13) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Best y = 'the the dog black'
P(y,d|x) = 5.6893074980508956e-05


100% (13 of 13) |#########################| Elapsed Time: 0:00:00 Time: 0:00:00
N/A% (0 of 13) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Best y = 'the dog black'
P(y,d|x) = 0.06311787343618726


100% (13 of 13) |#########################| Elapsed Time: 0:00:00 Time: 0:00:00
N/A% (0 of 13) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Best y = 'the dog black'
P(y,d|x) = 0.06311787343618726


100% (13 of 13) |#########################| Elapsed Time: 0:00:00 Time: 0:00:00
N/A% (0 of 13) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Best y = 'the dog black'
P(y,d|x) = 0.06311787343618726


100% (13 of 13) |#########################| Elapsed Time: 0:00:00 Time: 0:00:00


Best y = 'the dog black'
P(y,d|x) = 0.06311787343618726


In [750]:
for k,v in w_test.items():
    print(k,v)

top 0.339644069552
span:lhs:src:3 -242.310009901
span:lhs:tgt:3 -8.27550845038
type:binary -1.31674535477
type:monotone -1.32255120756
span:rhs:src-lc:1 -0.98043545535
span:rhs:src-rc:2 -1.34603581597
span:rhs:tgt-lc:0 -0.215293861632
span:rhs:tgt-rc:3 0.00708555517364
type:inverted -1.32304243941
span:rhs:tgt-lc:2 0.00876599015752
span:rhs:tgt-rc:1 -1.31985806446
type:deletion-slc -1.3183715301
span:rhs:src-lc:0 -1.31760264275
span:rhs:src-rc:3 -1.78274884
span:rhs:tgt-lc:1 -1.31659681065
span:rhs:tgt-rc:2 0.000137932466083
span:rhs:src-lc:2 -1.34159422451
span:rhs:src-rc:1 -0.982794087279
span:rhs:tgt-lc:3 0.00122735528093
span:rhs:tgt-rc:0 -3.86909719928
type:terminal -15.4610406697
type:deletion -45.0097030504
del:le -45.0108147045
span:rhs:src:1 -1.81592725594
span:rhs:tgt:0 -45.0105104179
span:lhs:src:1 -1.81156755447
span:lhs:tgt:0 -45.0028041471
type:translation -1.81407245987
trans:le/the -1.81554771231
span:rhs:tgt:1 -15.4594418747
span:lhs:tgt:1 -15.4643383349
type:deletion-

In [743]:
import progressbar
from itertools import chain

class SGD:
    """
    Class used to run sgd on a corpus.
    """
    def __init__(self, w_init, parse_dict=dict()):
        self.w = w_init
        self.parse_dict = parse_dict
    
    
    def update_w(self, wmap, expected_features_D_xy, expected_features_Dn_x, delta=0.1):
        w_new = defaultdict(float)

        for rule in chain(expected_features_D_xy, expected_features_Dn_x):
            for feature in chain(expected_features_D_xy[rule], expected_features_Dn_x[rule]):
                w_new[feature] = wmap[feature] + delta * (expected_features_D_xy[rule][feature] - 
                                                          expected_features_Dn_x[rule][feature])
                
        return w_new

    def epoch(self, minibatch, delta, 
              parses=[], sparse=False):
        """
        Performs stochastic gradient ascent on the weights vector w.
        """
        w = defaultdict(float)
        for i, (src_sent, tgt_sent) in enumerate(minibatch):
            
            if not parses:
                # get all required forests and fsa's
                target_forest, ref_forest, src_fsa = parse_forests(src_sent, tgt_sent, lexicon)
            
            else:
                target_forest, ref_forest, src_fsa = parses[i]
                
            # make sure i is the sentence index! not the minibatch index
            #target_forest, ref_forest, src_fsa = load_parses_separate('../parses/', i) 
                        
            ### D_n(x) ###

            tgt_edge2fmap, _ = featurize_edges(target_forest, src_fsa,
                                               sparse_del=sparse, sparse_ins=sparse, sparse_trans=sparse)

            # recompute edge weights
            tgt_edge_weights = {edge: weight_function(edge, tgt_edge2fmap[edge], self.w) 
                                for edge in target_forest}

            # compute inside and outside
            tsort = top_sort(target_forest)
            root = Nonterminal("D_n(x)")
            I_tgt = inside_algorithm(target_forest, tsort, tgt_edge_weights)
            O_tgt = outside_algorithm(target_forest, tsort, tgt_edge_weights, I_tgt, root)

            # compute expected features
            expected_features_Dn_x = expected_feature_vector(target_forest, I_tgt, O_tgt, tgt_edge2fmap)

            
            ### D(x,y) ###

            ref_edge2fmap, _ = featurize_edges(ref_forest, src_fsa,
                                               sparse_del=sparse, sparse_ins=sparse, sparse_trans=sparse)

            # recompute edge weights
            ref_edge_weights = {edge: weight_function(edge, ref_edge2fmap[edge], self.w) 
                                for edge in ref_forest}


            # compute inside and outside
            tsort = top_sort(ref_forest)
            root = Nonterminal("D(x,y)")
            I_ref = inside_algorithm(ref_forest, tsort, ref_edge_weights)
            O_ref = outside_algorithm(ref_forest, tsort, ref_edge_weights, I_ref, root)

            # compute expected features
            expected_features_D_xy = expected_feature_vector(ref_forest, I_ref, O_ref, ref_edge2fmap)

            # optain new w
            w_new = update_w(w, expected_features_D_xy, expected_features_Dn_x, delta=delta)
            
            for feature, value in w_new.items():
                w[feature] += 1.0 / len(minibatch) * value
        # update w
        self.w = w

In [744]:
w_init = defaultdict(float)
for feature in fset:
    w_init[feature] = 0.06
sgd = SGD(w_init)

fr_sent = 'le chien noir'
en_sent = 'the black dog'
minibatch = [(fr_sent, en_sent)]
lexicon = make_lexicon_ALT(fr_sent, en_sent)
parses = parse_forests(fr_sent, en_sent, lexicon)

n = 3
bar = progressbar.ProgressBar(max_value=n)
for i in range(n):
    bar.update(i)
    sgd.epoch(minibatch, 0.06, parses=[parses])
    bar.update(i+1)

w_new = sgd.w

N/A% (0 of 3) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

KeyboardInterrupt: 

In [188]:
for k,v in w_new.items():
    print(k,v)

top 2.7077020112348422e-173
span:lhs:src:3 -6.433031916918697e-228
span:lhs:tgt:3 -2.988060833832837e-172
type:binary -1.537226172264425e-279
type:monotone -1.537226172264425e-279
span:rhs:src-lc:1 -5.1916922859004495e-224
span:rhs:src-rc:2 -3.677435003397298e-289
span:rhs:tgt-lc:0 -4.0329128633699385e-285
span:rhs:tgt-rc:3 0.0
type:inverted -1.537226172264425e-279
span:rhs:tgt-lc:2 0.0
span:rhs:tgt-rc:1 -1.537226172264425e-279
type:deletion-slc -1.537226172264425e-279
span:rhs:src-lc:0 -1.537226172264425e-279
span:rhs:src-rc:3 -3.819921718785697e-284
span:rhs:tgt-lc:1 -1.537226172264425e-279
span:rhs:tgt-rc:2 0.0
span:rhs:src-lc:2 -3.677435003397299e-289
span:rhs:src-rc:1 -5.1916922859004495e-224
span:rhs:tgt-lc:3 0.0
span:rhs:tgt-rc:0 -1.2591361661413968e-223
type:terminal -2.301228817854101e-223
type:deletion -1.1490500340654827e-172
span:rhs:src:1 -5.569582552086917e-173
span:rhs:tgt:0 -1.1490500340654827e-172
span:lhs:src:1 -5.569582552086917e-173
span:lhs:tgt:0 -1.149050034065482

# Prediction

Finally, at prediction time we have a big problem, because
\begin{align}
y^\star =    \arg,\max_y P(y|x, n)
\end{align}
is intractable as proven by [Sima'an (1996)](http://www.aclweb.org/anthology/C/C96/C96-2215.pdf).
Thus we typically approximate that criterion by
\begin{align}
y^\star =  \text{yield}_\Delta \left(  \arg,\max_d P(y, d|x, n) \right)
\end{align}
which is often termed *Viterbi decoding*.

All this takes is a forest respresenting all translation candidates, i.e. \\(\mathcal D_n(x)\\) and a the Viterbi algorithm. Because in this project you need to implement *inside*, there's little point in programming Viterbi, you can simply use Inside for that. Check our [slides](https://uva-slpl.github.io/nlp2/resources/slides/crf.pdf) again.




In [301]:
def viterbi(forest: CFG, tsort: list, edge_weights: dict, inside: dict, root: Symbol) -> dict:
    """Returns the outside weight of each node"""
    Q = deque([root])
    V = list()
    while Q:
        symbol = Q.popleft()
        incoming = forest.get(symbol)
        weights = [0.0]*len(incoming)
        for i, edge in enumerate(incoming):
            weights[i] = edge_weights[edge]
            for u in edge.rhs: # u in tail(e)
                weights[i] *= inside[u] # TODO: change to log-sum-exp
        weight, selected = max(zip(weights, incoming), key=lambda xy: xy[0])
        for sym in selected.rhs:
            if not sym.is_terminal():
                Q.append(sym)
        V.append(selected)    
    return V

In [ ]:
def write_derrivation(d):
    derivation_as_fsa = libitg.forest_to_fsa(CFG(d), d[0].lhs)
    candidates = libitg.enumerate_paths_in_fsa(derivation_as_fsa)
    return candidates
candidates = write_derrivation(d)
print(candidates)

In [460]:
from nltk import Tree
from collections import defaultdict

def make_nltk_tree(derivation):
    """return a nlt Tree object based on the derivation (list or tuple of Rules)."""
    d = defaultdict(None, ((r.lhs, r.rhs) for r in derivation))
    
    def make_tree(lhs):
        return Tree(lhs.obj(), (child if child not in d else make_tree(child) for child in d[lhs]))
    
    return make_tree(derivation[0].lhs)

t = make_nltk_tree(d)
t.draw()

# Processing data

In [313]:
from math import nan
import string

def preprocess_lexicon(path='data/lexicon', k=5, null=5, remove_punct=True):
    f = open(path, 'r')
    
    ch_en_ = defaultdict(lambda: defaultdict(float))
    en_ch_ = defaultdict(lambda: defaultdict(float))

    for line in f:
        ch, en, p_en_given_ch, p_ch_given_en = line.split()
        # for use in the parsing we replace <NULL> with -EPS-
        if ch == '<NULL>':
            ch = '-EPS-' 
        if en == '<NULL>':
            en = '-EPS-' 
        ch_en_[ch][en] = float(p_en_given_ch) if not p_en_given_ch == 'NA' else nan # perhaps something tiny like 10e-20
        en_ch_[en][ch] = float(p_ch_given_en) if not p_ch_given_en == 'NA' else nan # perhaps something tiny like 10e-20
    f.close()
    
    ch_en = defaultdict(lambda: defaultdict(float))
    for ch in ch_en_.keys():
        en_punct = string.punctuation
        srtd = sorted(ch_en_[ch].items(), key=lambda xy: xy[1])
        if ch == '-EPS-':
            if remove_punct:
                # when we do not want to insert punctuation from EPS
                srtd = [(en, p) for en, p in srtd if en not in en_punct]
            ch_en['-EPS-'] = {en: p for en, p in srtd[-null:]}
        else:
            ch_en[ch] = {en: p for en, p in srtd[-k:]}


    en_ch = defaultdict(lambda: defaultdict(float))
    for en in en_ch_.keys():
        ch_punct = "[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？? 、~@#￥%……&*（）：；《）《》“”()»〔〕-]+"
        srtd = sorted(en_ch_[en].items(), key=lambda xy: xy[1])
        if en == '-EPS-=':
            if remove_punct:
                # when we do not want to insert punctuation from EPS
                srtd = [(ch, p) for ch, p in srtd if ch not in ch_punct]
            en_ch['-EPS-'] = {ch: p for ch, p in srtd[-null:]}
        else:
            en_ch[en] = {ch: p for ch, p in srtd[-k:]}
            
    full_en_ch = en_ch_
    full_ch_en = ch_en_
    return ch_en, en_ch, full_en_ch, full_ch_en


In [141]:
ch_en, en_ch, full_en_ch, full_ch_en = preprocess_lexicon()

In [142]:
print(len(ch_en))
print(len(en_ch))
print(en_ch['spring'])
print(ch_en['-EPS-'])
print(en_ch['-EPS-'])

print(full_en_ch['the']['-EPS-']) # deletion of 'however' (given 'however' produce EPS)
print(full_en_ch['-EPS-']['春天']) # insertion of '春天' (given EPS produce '春天')

13960
10512
{'开课': 0.030781041830778122, '春卷': 0.052335865795612335, '明年': 0.05507362633943558, '是': 0.11269595474004745, '春天': 0.5166555643081665}
{'it': 0.06706574559211731, 'to': 0.07063718140125275, 'a': 0.08496548235416412, 'i': 0.10233193635940552, 'the': 0.11601653695106506}
{'了': 0.02192385494709015, '?': 0.04991278424859047, '我': 0.10082501918077469, '的': 0.25213390588760376, '。': 0.45774778723716736}
nan
3.0006491645135287e-31


In [143]:
def read_data_dev(path='data/dev1.zh-en', max_sents=5):
    f = open(path, 'r')
    corpus = dict()
    for k, line in enumerate(f):
        if k + 1 > max_sents:
            break
        sents = line[:-1].split('|||')
        ch = sents[0]
        translations = list()
        for en in sents[1:]:
            translations.append(en)
        corpus[ch] = translations
    return corpus

def read_data(path='data/training.zh-en', max_sents=5):
    f = open(path, 'r')
    corpus = list()
    for k, line in enumerate(f):
        if k + 1 > max_sents:
            break
        ch, en = line[:-1].split('|||') # line[:-1] to remove '\n' character
        corpus.append((ch, en))
    return corpus

def make_lexicon(ch_sent, ch_en, en_ch):
    """
    Given a chinese sentence produces a lexicon of possible translation as dictionary
    Format: chinese character -> {top 5 english translations}
    :param ch_sent: a chinese sentence as string (e.g. '在 门厅 下面 。 ')
    :param ch_sent: a chinese sentence as string (e.g. 'it 's just down the hall .')
    """
    lexicon = defaultdict(set)
    lexicon['-EPS-'].update(ch_en['-EPS-'])
    for char in ch_sent.split():
        lexicon[char].update(ch_en[char])
    return lexicon

def make_lexicon_ALT(ch_sent, en_sent):
    """
    Given a chinese sentence produces a lexicon of possible translation as dictionary
    Format: chinese character -> {all english words in training sentence}
    """
    lexicon = defaultdict(set)
    lexicon['-EPS-'].update(en_sent.split())
    for char in ch_sent.split():
        lexicon[char].update(en_sent.split())
        lexicon[char].add('-EPS-')
    return lexicon

In [144]:
corpus = read_data()
print(corpus)
ch_sent = '一 跳 一 跳 的 痛 。 '
lexicon = make_lexicon(ch_sent, ch_en, en_ch)
print(lexicon)

[('在 门厅 下面 。 ', " it 's just down the hall ."), ('我 这 就 给 您 拿 一些 。 ', " i 'll bring you some now ."), ('如果 您 还 有 什么 需要 , 尽管 告诉 我 。 ', ' if there is anything else you need , just let me know .'), ('不用 担心 那个 。 ', ' no worry about that .'), ('我 要 买 它 , 你 不 需要 把 它 包 起来 。 ', " i 'll take it and you need not wrap it up .")]
defaultdict(<class 'set'>, {'-EPS-': {'it', 'the', 'i', 'a', 'to'}, '一': {'one', '.', 'of', '-EPS-', 'a'}, '跳': {'it', "'s", 'pain', '-EPS-', 'throbbing'}, '的': {'the', '.', 'of', '-EPS-', 'a'}, '痛': {'i', 'pain', '-EPS-', "'ve", 'hurts'}, '。': {',', 'i', '.', '-EPS-', 'a'}})


In [145]:
def test():
    ch_sent, en_sent = corpus[1]
    print(ch_sent)
    print(en_sent)
#     lexicon = make_lexicon(ch_sent, ch_en, en_ch)
    lexicon = make_lexicon_ALT(ch_sent, en_sent)

    print(lexicon)
    src_fsa = libitg.make_fsa(ch_sent)
    print(src_fsa)
    src_cfg = libitg.make_source_side_itg(lexicon)
    print(src_cfg)
    forest = libitg.earley(src_cfg, src_fsa, 
                           start_symbol=Nonterminal('S'), 
                           sprime_symbol=Nonterminal("D(x)"))
    print('forest: {}'.format(len(forest)))

    projected_forest = libitg.make_target_side_itg(forest, lexicon)
    print('projected forest: {}'.format(len(projected_forest)))
    
    tgt_fsa = libitg.make_fsa(en_sent)
    print(tgt_fsa)
    ref_forest = libitg.earley(projected_forest, tgt_fsa, 
                               start_symbol=Nonterminal("D(x)"), 
                               sprime_symbol=Nonterminal('D(x,y)'))
    print('ref forest: {}'.format(len(ref_forest)))

    print('pass1')
    length_fsa = libitg.LengthConstraint(len(en_sent), strict=False)
    print('pass2')

    target_forest = libitg.earley(projected_forest, length_fsa, 
                                  start_symbol=Nonterminal("D(x)"), 
                                  sprime_symbol=Nonterminal("D_n(x)"))
    print('target forest: {}'.format(len(target_forest)))
    print('pass3')

    return target_forest, ref_forest, src_fsa

In [146]:
ch_target_forest, ch_ref_forest, ch_src_fsa = test()

我 这 就 给 您 拿 一些 。 
 i 'll bring you some now .
defaultdict(<class 'set'>, {'-EPS-': {'i', '.', 'some', "'ll", 'you', 'now', 'bring'}, '我': {'i', '.', 'some', "'ll", '-EPS-', 'you', 'now', 'bring'}, '这': {'i', '.', 'some', "'ll", '-EPS-', 'you', 'now', 'bring'}, '就': {'i', '.', 'some', "'ll", '-EPS-', 'you', 'now', 'bring'}, '给': {'i', '.', 'some', "'ll", '-EPS-', 'you', 'now', 'bring'}, '您': {'i', '.', 'some', "'ll", '-EPS-', 'you', 'now', 'bring'}, '拿': {'i', '.', 'some', "'ll", '-EPS-', 'you', 'now', 'bring'}, '一些': {'i', '.', 'some', "'ll", '-EPS-', 'you', 'now', 'bring'}, '。': {'i', '.', 'some', "'ll", '-EPS-', 'you', 'now', 'bring'}})
states=9
initial=0
final=8
arcs=8
origin=0 destination=1 label=我
origin=1 destination=2 label=这
origin=2 destination=3 label=就
origin=3 destination=4 label=给
origin=4 destination=5 label=您
origin=5 destination=6 label=拿
origin=6 destination=7 label=一些
origin=7 destination=8 label=。
[S] ||| [X]
[X] ||| [X] [X]
[X] ||| '-EPS-'
[X] ||| '我'
[X] ||| '这'
[X

KeyboardInterrupt: 

In [147]:
w_init = defaultdict(float)
for feature in fset:
    w_init[feature] = 0.06
w_new = sgd_func(1, 0.6, w_init, ch_target_forest, ch_ref_forest, ch_src_fsa, sparse=False)

print(w_new)

NameError: name 'ch_target_forest' is not defined

In [ ]:
ch_edge2fmap, _ = featurize_edges(ch_target_forest, ch_src_fsa,
                                  sparse_del=False, sparse_ins=False, sparse_trans=False)

ch_estimated_edge_weights = {edge: weight_function(edge, ch_edge2fmap[edge], w_new) for edge in ch_target_forest}
ch_tsort = top_sort(ch_target_forest)
ch_I = inside_algorithm(ch_ref_forest, ch_tsort, ch_estimated_edge_weights)
ch_d = viterbi(ch_target_forest, ch_I, Span(Nonterminal('D(x)'), 0, 2), ch_estimated_edge_weights)
ch_tsort = top_sort(ch_target_forest)
ch_d = Viterbi(ch_target_forest, ch_tsort, ch_estimated_edge_weights, I, Span(Nonterminal('D(x)'), 0, 2))
write(ch_d)

# Saving and loading parses

In [ ]:
# import cPickle as pickle
import pickle

def parse_forests(src_sent, tgt_sent, lexicon):
    """
    Parses the forests needed for epoch
    """
    src_fsa = libitg.make_fsa(src_sent)
    src_cfg = libitg.make_source_side_itg(lexicon)
    forest = libitg.earley(src_cfg, src_fsa, 
                           start_symbol=Nonterminal('S'), 
                           sprime_symbol=Nonterminal("D(x)"))
    
    projected_forest = libitg.make_target_side_itg(forest, lexicon)
    
    tgt_fsa = libitg.make_fsa(tgt_sent)
    ref_forest = libitg.earley(projected_forest, tgt_fsa, 
                               start_symbol=Nonterminal("D(x)"), 
                               sprime_symbol=Nonterminal('D(x,y)'),
                               eps_symbol=Nonterminal('-EPS-'))
    
    length_fsa = libitg.LengthConstraint(len(tgt_sent), strict=False)
    target_forest = libitg.earley(projected_forest, length_fsa, 
                                  start_symbol=Nonterminal("D(x)"), 
                                  sprime_symbol=Nonterminal("D_n(x)"))
    
    return target_forest, ref_forest, src_fsa


def save_parses(corpus, savepath):
    """
    Parses all sentences in corpus and saves a triple of needed ones in (huge) dictionary
    indexed by sentence number in corpus as pkl object at savepath.

    :corpus: a list of tuples [(chinese sentence, english sentence)] 
    :saves: parse_dict: sentence number -> (target_forest, ref_forest, scr_fsa)   
    """
    parse_dict = dict() 
    for i, (ch_sent, en_sent) in enumerate(corpus):
        lexicon = make_lexicon_ALT(ch_sent, en_sent)

        parses = parse_forests(ch_sent, en_sent, lexicon)
        
        parse_dict[i] = parses
    
    f = open(savepath + 'parse_dict.pkl', 'wb')
    pickle.dump(parse_dict, f, protocol=4)
    f.close()

def save_parses_separate(corpus, savepath):
    """
    For each sentence k in corpus we parse and save the triple of needed parses 
    as pkl object at savepath/parses-k.pkl.

    :corpus: a list of tuples [(chinese sentence, english sentence)] 
    :saves: parses-k = (target_forest, ref_forest, scr_fsa) for each k in 0,..,len(corpus)
    """
    for k, (ch_sent, en_sent) in enumerate(corpus):
        lexicon = make_lexicon_ALT(ch_sent, en_sent)

        parses = parse_forests(ch_sent, en_sent, lexicon)
        
        f = open(savepath + 'parses-{}.pkl'.format(k), 'wb')
        pickle.dump(parses, f, protocol=4)
        f.close()
    
def load_parses(savepath):
    """
    Loads and returns a parse_dict as saved by load_parses.
    """
    f = open(savepath + 'parse_dict.pkl', 'rb')
    parse_dict = pickle.load(f)
    f.close()
    return parse_dict

def load_parses_separate(savepath, k):
    """
    Loads and returns parses as saved by load_parses_separate
    """
    f = open(savepath + 'parses-{k}.pkl', 'rb')
    parses = pickle.load(f)
    f.close()
    return parses

In [ ]:
save_parses(corpus[:1], '')

parse_dict = load_parses('')
print(parse_dict[0])

In [ ]:
save_parses_separate(corpus[:3], '../parses/')

# parses = load_parses_separate('../parses/')

In [ ]:
w_init = defaultdict(float)
for feature in fset:
    w_init[feature] = 0.06
sgd = SGD(w_init, parse_dict)

for i in range(1):
    sgd.epoch([('le chien noir', 'the black dog')], 0.6, sparse=False)

w_new = sgd.w